##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 保存和恢复模型

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/save_and_load">     <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">     在 TensorFlow.org 上查看</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/keras/save_and_load.ipynb" class="_active_edit_href"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>
</td>
</table>

可以在训练期间和之后保存模型进度。这意味着模型可以从停止的地方恢复，避免长时间的训练。此外，保存还意味着您可以分享您的模型，其他人可以重现您的工作。在发布研究模型和技术时，大多数机器学习从业者会分享：

- 用于创建模型的代码
- 模型训练的权重 (weight) 和参数 (parameters) 。

共享数据有助于其他人了解模型的工作原理，并使用新数据自行尝试。

小心：TensorFlow 模型是代码，对于不受信任的代码，一定要小心。请参阅 [安全使用 TensorFlow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) 以了解详情。

### 选项

根据您使用的 API，可以通过多种方式保存 TensorFlow 模型。本指南使用 [tf.keras](https://tensorflow.google.cn/guide/keras)，这是一种在 TensorFlow 中构建和训练模型的高级 API。对于其他方式，请参阅[使用 SavedModel 格式指南](../../guide/saved_model.ipynb)以及[保存和加载 Keras 模型指南](https://tensorflow.google.cn/guide/keras/save_and_serialize)。

## 配置

### 安装并导入

安装并导入Tensorflow和依赖项：

In [3]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)


In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2023-11-23 17:13:52.311142: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-23 17:13:52.330021: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 17:13:52.330035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 17:13:52.330049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 17:13:52.333708: I tensorflow/core/platform/cpu_feature_g

2.14.0


In [3]:
tf.test.is_gpu_available(),tf.config.list_physical_devices('GPU')

2023-11-23 16:17:22.710348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 16:17:22.710729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 16:17:22.710801: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

(True,
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
  PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')])

s-bus-pci#L344-L355
2023-11-23 16:17:22.710900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 16:17:22.710950: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 16:17:22.711033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentat

### 获取示例数据集

为了演示如何保存和加载权重，您将使用 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。为了加快运行速度，请使用前 1000 个样本：

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 定义模型

首先构建一个简单的序列（sequential）模型：

In [3]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-11-23 17:14:00.071139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 17:14:00.071236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 17:14:00.084742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## 在训练期间保存模型（以 checkpoints 形式保存）

您可以使用经过训练的模型而无需重新训练，或者在训练过程中断的情况下从离开处继续训练。`tf.keras.callbacks.ModelCheckpoint` 回调允许您在训练*期间*和*结束*时持续保存模型。

### Checkpoint 回调用法

创建一个只在训练期间保存权重的 `tf.keras.callbacks.ModelCheckpoint` 回调：

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

这将创建一个 TensorFlow checkpoint 文件集合，这些文件在每个 epoch 结束时更新：

In [5]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

只要两个模型共享相同的架构，您就可以在它们之间共享权重。因此，当从仅权重恢复模型时，创建一个与原始模型具有相同架构的模型，然后设置其权重。

现在，重新构建一个未经训练的全新模型并基于测试集对其进行评估。未经训练的模型将以机会水平执行（约 10% 的准确率）：

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3587 - sparse_categorical_accuracy: 0.0590 - 136ms/epoch - 4ms/step
Untrained model, accuracy:  5.90%


然后从 checkpoint 加载权重并重新评估：

In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4235 - sparse_categorical_accuracy: 0.8770 - 25ms/epoch - 787us/step
Restored model, accuracy: 87.70%


In [16]:
import tensorflow.compat.v1 as tf1
#使用NewCheckpointReader来读取ckpt里的变量
 
#from tensorflow.python import pywrap_tensorflow
 
#checkpoint_path = os.path.join(model_dir, "model.ckpt")
 
reader = tf1.train.NewCheckpointReader(checkpoint_path) #tf.train.NewCheckpointReader
 
var_to_shape_map = reader.get_variable_to_shape_map()
 
print(type(var_to_shape_map))
print(var_to_shape_map)
 
for key in var_to_shape_map:
 
  print("tensor_name: ", key)
 
  print(reader.get_tensor(key))


<class 'dict'>
{'optimizer/_variables/8/.ATTRIBUTES/VARIABLE_VALUE': [10], 'optimizer/_variables/7/.ATTRIBUTES/VARIABLE_VALUE': [10], '_CHECKPOINTABLE_OBJECT_GRAPH': [], 'layer_with_weights-0/bias/.ATTRIBUTES/VARIABLE_VALUE': [512], 'optimizer/_iterations/.ATTRIBUTES/VARIABLE_VALUE': [], 'layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE': [10], 'optimizer/_learning_rate/.ATTRIBUTES/VARIABLE_VALUE': [], 'optimizer/_variables/1/.ATTRIBUTES/VARIABLE_VALUE': [784, 512], 'optimizer/_variables/2/.ATTRIBUTES/VARIABLE_VALUE': [784, 512], 'optimizer/_variables/6/.ATTRIBUTES/VARIABLE_VALUE': [512, 10], 'optimizer/_variables/3/.ATTRIBUTES/VARIABLE_VALUE': [512], 'optimizer/_variables/4/.ATTRIBUTES/VARIABLE_VALUE': [512], 'layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE': [784, 512], 'layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE': [512, 10], 'optimizer/_variables/5/.ATTRIBUTES/VARIABLE_VALUE': [512, 10]}
tensor_name:  optimizer/_variables/8/.ATTRIBUTES/VARIABLE_VALUE
[4.247857

### checkpoint 回调选项

回调提供了几个选项，为 checkpoint 提供唯一名称并调整 checkpoint 频率。

训练一个新模型，每五个 epochs 保存一次唯一命名的 checkpoint ：

In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt



Epoch 10: saving model to training_2/cp-0010.ckpt



Epoch 15: saving model to training_2/cp-0015.ckpt



Epoch 20: saving model to training_2/cp-0020.ckpt



Epoch 25: saving model to training_2/cp-0025.ckpt



Epoch 30: saving model to training_2/cp-0030.ckpt



Epoch 35: saving model to training_2/cp-0035.ckpt



Epoch 40: saving model to training_2/cp-0040.ckpt



Epoch 45: saving model to training_2/cp-0045.ckpt



Epoch 50: saving model to training_2/cp-0050.ckpt


现在，检查生成的检查点并选择最新检查点：

In [12]:
os.listdir(checkpoint_dir)

['cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.index']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

注：默认 TensorFlow 格式只保存最近的 5 个检查点。

要进行测试，请重置模型并加载最新检查点：

In [14]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4798 - sparse_categorical_accuracy: 0.8800 - 156ms/epoch - 5ms/step


Restored model, accuracy: 88.00%


## 这些文件是什么？

上述代码可将权重存储到[检查点](../../guide/checkpoint.ipynb)格式文件（仅包含二进制格式训练权重） 的合集中。检查点包含：

- 一个或多个包含模型权重的分片。
- 一个索引文件，指示哪些权重存储在哪个分片中。

如果您在一台计算机上训练模型，您将获得一个具有如下后缀的分片：`.data-00000-of-00001`

## 手动保存权重

要手动保存权重，请使用 `tf.keras.Model.save_weights`。默认情况下，`tf.keras`（尤其是 `Model.save_weights` 方法）使用扩展名为 `.ckpt` 的 TensorFlow [检查点](../../guide/checkpoint.ipynb)格式。要以扩展名为 `.h5` 的 HDF5 格式保存，请参阅[保存和加载模型](https://tensorflow.google.cn/guide/keras/save_and_serialize)指南。

In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4798 - sparse_categorical_accuracy: 0.8800 - 155ms/epoch - 5ms/step


Restored model, accuracy: 88.00%


## 保存整个模型

调用 `tf.keras.Model.save` 以将模型的架构、权重和训练配置保存在单个 `file/folder` 中。这样一来，您可以导出模型，以便在不访问原始 Python 代码的情况下使用它*。由于优化器状态已经恢复，可以从中断的位置恢复训练。

整个模型可以保存为两种不同的文件格式（`SavedModel` 和 `HDF5`）。TensorFlow `SavedModel` 格式是 TF2.x 中的默认文件格式。但是，模型能够以 `HDF5` 格式保存。下面详细介绍了如何以两种文件格式保存整个模型。

保存全功能模型会非常有用，您可以在 TensorFlow.js（[Saved Model](https://tensorflow.google.cn/js/tutorials/conversion/import_saved_model)、[HDF5](https://tensorflow.google.cn/js/tutorials/conversion/import_keras)）中加载它们，然后在网络浏览器中训练和运行，或者使用 TensorFlow Lite（[Saved Model](https://tensorflow.google.cn/lite/models/convert/#convert_a_savedmodel_recommended_)、[HDF5](https://tensorflow.google.cn/lite/models/convert/#convert_a_keras_model_)）转换它们以在移动设备上运行

*自定义对象（例如，子类化模型或层）在保存和加载时需要特别注意。请参阅下面的**保存自定义对象**部分。

### SavedModel 格式

SavedModel 格式是另一种序列化模型的方式。以这种格式保存的模型可以使用 `tf.keras.models.load_model` 还原，并且与 TensorFlow Serving 兼容。[SavedModel 指南](../../guide/saved_model.ipynb)详细介绍了如何 `serve/inspect` SavedModel。以下部分说明了保存和恢复模型的步骤。

In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5



 1/32 [..............................] - ETA: 9s - loss: 2.4671 - sparse_categorical_accuracy: 0.0312


26/32 [=======================>......] - ETA: 0s - loss: 1.2919 - sparse_categorical_accuracy: 0.6154


32/32 [==============================] - 0s 2ms/step - loss: 1.1801 - sparse_categorical_accuracy: 0.6520


Epoch 2/5


...

Epoch 5/5



 1/32 [..............................] - ETA: 0s - loss: 0.3043 - sparse_categorical_accuracy: 0.9062


27/32 [========================>.....] - ETA: 0s - loss: 0.1683 - sparse_categorical_accuracy: 0.9606


32/32 [==============================] - 0s 2ms/step - loss: 0.1728 - sparse_categorical_accuracy: 0.9590


INFO:tensorflow:Assets written to: saved_model/my_model/assets


SavedModel 格式是一个包含 protobuf 二进制文件和 TensorFlow 检查点的目录。检查保存的模型目录：

In [17]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model


assets	keras_metadata.pb  saved_model.pb  variables


从保存的模型重新加载一个新的 Keras 模型：

In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 dense_10 (Dense)            (None, 512)               401920    


 dropout_5 (Dropout)         (None, 512)               0         


 dense_11 (Dense)            (None, 10)                5130      


Total params: 407,050


Trainable params: 407,050


Non-trainable params: 0


_________________________________________________________________


使用与原始模型相同的参数编译恢复的模型。尝试使用加载的模型运行评估和预测：

In [19]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4184 - sparse_categorical_accuracy: 0.8590 - 164ms/epoch - 5ms/step


Restored model, accuracy: 85.90%

 1/32 [..............................] - ETA: 2s


32/32 [==============================] - 0s 1ms/step


(1000, 10)


### HDF5 格式

Keras使用 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 标准提供了一种基本的保存格式。 

In [20]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5



 1/32 [..............................] - ETA: 9s - loss: 2.4336 - sparse_categorical_accuracy: 0.0312


25/32 [======================>.......] - ETA: 0s - loss: 1.2285 - sparse_categorical_accuracy: 0.6450


32/32 [==============================] - 0s 2ms/step - loss: 1.1226 - sparse_categorical_accuracy: 0.6750


Epoch 2/5


...


32/32 [==============================] - 0s 2ms/step - loss: 0.2025 - sparse_categorical_accuracy: 0.9560


Epoch 5/5



 1/32 [..............................] - ETA: 0s - loss: 0.2466 - sparse_categorical_accuracy: 0.8750


27/32 [========================>.....] - ETA: 0s - loss: 0.1521 - sparse_categorical_accuracy: 0.9676


32/32 [==============================] - 0s 2ms/step - loss: 0.1622 - sparse_categorical_accuracy: 0.9640


现在，从该文件重新创建模型：

In [21]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 dense_12 (Dense)            (None, 512)               401920    


 dropout_6 (Dropout)         (None, 512)               0         


 dense_13 (Dense)            (None, 10)                5130      


Total params: 407,050


Trainable params: 407,050


Non-trainable params: 0


_________________________________________________________________


检查其准确率（accuracy）：

In [22]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4360 - sparse_categorical_accuracy: 0.8600 - 157ms/epoch - 5ms/step


Restored model, accuracy: 86.00%


Keras 通过检查模型的架构来保存这些模型。这种技术可以保存所有内容：

- 权重值
- 模型的架构
- 模型的训练配置（您传递给 `.compile()` 方法的内容）
- 优化器及其状态（如果有）（这样，您便可从中断的地方重新启动训练）

Keras 无法保存 `v1.x` 优化器（来自 `tf.compat.v1.train`），因为它们与检查点不兼容。对于 v1.x 优化器，您需要在加载-失去优化器的状态后，重新编译模型。


### 保存自定义对象

如果您正在使用 SavedModel 格式，则可以跳过此部分。HDF5 和 SavedModel 之间的主要区别在于，HDF5 使用对象配置来保存模型架构，而 SavedModel 则保存执行计算图。因此，SavedModel 能够在不需要原始代码的情况下保存自定义对象，如子类模型和自定义层。

要将自定义对象保存到 HDF5，您必须执行以下操作：

1. 在您的对象中定义一个 `get_config` 方法，并且可以选择定义一个 `from_config` 类方法。
    - `get_config(self)` 返回重新创建对象所需的参数的 JSON 可序列化字典。
    - `from_config(cls, config)` 使用从 `get_config` 返回的配置来创建一个新对象。默认情况下，此函数将使用配置作为初始化 kwarg (`return cls(**config)`)。
2. 加载模型时将对象传递给 `custom_objects` 参数。参数必须是将字符串类名映射到 Python 类的字典。例如 `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

有关自定义对象和 `get_config` 的示例，请参阅[从头开始编写层和模型](https://tensorflow.google.cn/guide/keras/custom_layers_and_models)教程。
